# Import packages

In [13]:
import pandas as pd
import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status
from dft_descriptors import prepocessing as pp
import logging
logging.basicConfig(level=logging.INFO)

# Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [14]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz


TimeoutError: [Errno 60] Operation timed out

# Provide smiles string(s)
and copy into them into my_smiles variable. It's a list of strings (a Python list can be of any size), the smiles are extracted for the csv resulting form the step 2 numbering the molecules.

In [3]:
react_nb = pd.read_csv("../data_csv/Data_test10222021.csv", sep=',')
import numpy as np
from rdkit import Chem
df = pd.read_csv('../data_csv/Data_test10222021.csv', sep = ',')
df = df[df['Ligand effectif'].notna()]

In [5]:
unik_lig = [pp.dict_ligand[i] for i in np.unique(df["Ligand effectif"])]
can_unik_lig = np.unique([Chem.CanonSmiles(smi) for smi in unik_lig])
my_smiles = can_unik_lig
can_unik_lig
# determine if ligand is NHC, Phosphine, Diphosphine or other
def lig_type(list_lig):
    lig_type = []
    for smi in list_lig:
        m = Chem.MolFromSmiles(smi)
        if m.HasSubstructMatch(Chem.MolFromSmiles('N[C]N')):
            typ = 'NHC'
            if m.HasSubstructMatch(Chem.MolFromSmiles('P')):
                typ = 'mixed'
        elif m.HasSubstructMatch(Chem.MolFromSmiles('P')):
            typ = 'Phos'
            if len(m.GetSubstructMatches(Chem.MolFromSmiles('P'))) == 2:
                typ = 'DiPhos'
                if len(m.GetSubstructMatches(Chem.MolFromSmiles('P'))) > 2:
                    typ = 'mixed Phos'
        else:
            typ = 'other'
        lig_type.append(typ)
    return lig_type
nhc = [can_unik_lig[i] for i in range(len(can_unik_lig)) if lig_type(can_unik_lig)[i] == 'NHC']
nhc

['CC(C)(C)N1[C]N(C(C)(C)C)C=C1',
 'CC(C)(C)N1[C]N(C(C)(C)C)CC1',
 'CC(C)(C)N1[C]N(CN2[C]N(C(C)(C)C)C=C2)C=C1',
 'CC(C)N1[C]N(C(C)C)C=C1',
 'CC(C)N1[C]N(CN2[C]N(C(C)C)C=C2)C=C1',
 'CC(C)c1cccc(C(C)C)c1N1[C]N(c2c(C(C)C)cccc2C(C)C)C=C1',
 'CC(C)c1cccc(C(C)C)c1N1[C]N(c2c(C(C)C)cccc2C(C)C)CC1',
 'CC1=C(C)N(c2c(C(C)C)cccc2C(C)C)[C]N1c1c(C(C)C)cccc1C(C)C',
 'CCCCN1[C]N(c2cccc(N3[C]N(CCCC)c4ccccc43)n2)c2ccccc21',
 'CN1[C]N(CN2[C]N(C)C=C2)C=C1',
 'Cc1cc(C)c(N2[C]N(c3c(C)cc(C)cc3C)C=C2)c(C)c1',
 'Cc1cc(C)c(N2[C]N(c3c(C)cc(C)cc3C)CC2)c(C)c1',
 '[C]1N(C23CC4CC(CC(C4)C2)C3)C=CN1C12CC3CC(CC(C3)C1)C2',
 '[C]1N(C2C3CC4CC(C3)CC2C4)C=CN1C1C2CC3CC(C2)CC1C3',
 '[C]1N(C2CCCCC2)C=CN1C1CCCCC1',
 '[C]1N(CN2[C]N(C3CCCCC3)C=C2)C=CN1C1CCCCC1']

# Initialize the molecules and generate conformations
We will use some for loops to loop over the smiles, only one conformation will be taken for each molecules in this work

In [6]:
mols = []
for smile in nhc:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC(N1C=CN([C]1)C(C)(C)C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC(N1CCN([C]1)C(C)(C)C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC(N1C=CN([C]1)CN1C=CN([C]1)C(C)(C)C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: CC(N1C=CN([C]1)C(C)C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: C

###### Draw some molecules

In [8]:
#mols[0].draw(2)

# Create gaussian job for each conformation

In [9]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C11H20N2_7540_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C11H22N2_c5d7_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


...

C23H32N2_7ca4_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C15H24N2_9a3b_conf_0


INFO:aqc_utils.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


C19H28N4_583c_conf_0


In [10]:
sm.get_job_stats(split_by_can=True)

status,created,done,failed,incomplete,uploaded
can,,,,,
BrB(Br)Br,0,0,0,0,1
Br[Mg]C1CCCCC1,0,0,0,0,1
Br[Mg]Cc1ccccc1,0,0,0,0,1
Br[Mg]c1c(C)cc(cc1C)C,0,0,0,1,0
C(Oc1ccccc1)Oc1ccccc1,0,0,1,0,0
...,...,...,...,...,...
[Rb+],0,0,0,0,1
[Sr+2],0,0,0,0,1
[Y+3],0,0,0,0,1


In [11]:
sm.submit_jobs()

INFO:aqc_utils.slurm_manager:Submitting 16 jobs.
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)


Submitted batch job 10441


INFO:aqc_utils.slurm_manager:Submitted job 8d4e225e7fae1fe2e931f937179a6105, job_id: 10441.


Submitted batch job 10442


INFO:aqc_utils.slurm_manager:Submitted job a324cf6d41d1ad5cd07f7eeae4d94a63, job_id: 10442.


...

INFO:aqc_utils.slurm_manager:Submitted job 29dc634d0401b34e44b7a572b609ae6f, job_id: 10454.


Submitted batch job 10455


INFO:aqc_utils.slurm_manager:Submitted job 725198f8f67b773b9e7244c05a33838b, job_id: 10455.


Submitted batch job 10456


INFO:aqc_utils.slurm_manager:Submitted job f599b9d8ba83ca2ca87ef376c8ea95dc, job_id: 10456.


In [12]:
sm.squeue()

,RUNNING
jobs,16


In [90]:
for job in sm.get_jobs():
    if str(sm.get_jobs()[job].status) == 'slurm_status.done':
        print(sm.get_jobs()[job].base_name)

C14H18OSi_95fa_conf_0
C11H10O2_d7c7_conf_0
C11H10O_9eb4_conf_0
C11H17NO_9080_conf_0
C11H17NO_d167_conf_0
C10H11NO_10ad_conf_0
C12H12O_d53c_conf_0
C13H14O_eb43_conf_0


In [84]:
job

'6625625ec8945f514e3c68c6d47152bb'

In [85]:
job_all = sm.get_jobs()[job]

In [33]:
sm.get_job_stats()

status,done,failed,incomplete,uploaded
jobs,1,338,1,19


In [87]:
str(job_all.status)

'slurm_status.created'

In [88]:
job_all.base_name

'C12H13NO2_b9f2_conf_0'